<h1>Hackathon Dunnhumby</h1>

<b><u>Problem Statement:</u></b>

<i>A cosmetic product company wants to reward its loyal customers by giving them a heavy discount on the products they buy repeatedly. Probability of buying these products on the next visit is computed using a heuristic. Higher probability scores meant higher relevancy. Based on the scores the company wants to allocate the most relevant set of products to customers.</i>

<b><u>Objective:</u></b>

<i>The objective of the hackathon is to allocate the most relevant set of products to each customer by maximizing total relevancy. You should use the column “relevancy_score” of <b>Relevancy_table</b> to get relevancy of products for customers.</i>

<b><u>Constraints:</u></b>

1. Due to budget constraints, there is fixed volume of each product. For instance, product “5650512” cannot be allocated to more than 150 customers. Use the “Volume” column of the Products table.

2. A customer can get maximum 8 products and minimum 3 products. Drop all the customers who qualify for less than 3 products. 

3. There are some set of products which cannot be assigned together (e.g. product “5649565” and “5649646” cannot be given together to any customer). You can get this list in the Exclusion table.

4. All the products allocated to a customer should be distinct (i.e. the same product cannot be allocated twice to the same customer)

<i><b><u>Making necessary imports</u></b></i>

In [60]:
import pandas as pd

<b><u><i>Load the Relevancy Table</i></u></b>

In [111]:
data=pd.read_csv("Relevency_table.csv")
data

,customers,product,relevancy_score
0,A10001,5649565,0.293978
1,A10001,5649585,0.076184
2,A10001,5649607,0.312285
3,A10001,5649625,0.113652
4,A10001,5649630,0.108481
...,...,...,...
48411,A10787,5649878,0.008794
48412,A10787,5649900,0.002027
48413,A10787,5649925,0.002392
48414,A10787,5649963,0.002319


<i><b><u>Sorting the table in the ascending order of customers and then descending order of relevancy score for each customer</u></b></i>

In [112]:
data=data.sort_values(by=["customers","relevancy_score"],ascending=[True,False])
data

,customers,product,relevancy_score
91,A10001,5650743,0.646916
43,A10001,5649965,0.608653
11,A10001,5649679,0.587336
53,A10001,5650455,0.581182
54,A10001,5650462,0.575269
...,...,...,...
48408,A10787,5649836,0.002269
48405,A10787,5649677,0.002081
48412,A10787,5649900,0.002027
48409,A10787,5649845,0.002007


<u><i><b>Loading the Exclusion Table</b></i></u>

In [63]:
ex=pd.read_csv("Exclusion.csv")
ex

,product1,product2
0,5649565,5649646
1,5649585,5649910
2,5649585,5649921
3,5649607,5649931
4,5649607,5649929
...,...,...
215,5650785,5650772
216,5650785,5650777
217,5650785,5650783
218,5650785,5650677


<u><i><b>Making matrix to access exclusion of two products</b></i></u>

In [64]:
df=pd.crosstab(ex.product1,ex.product2)
df = df.reindex(index = df.columns.union(df.index), columns=df.columns.union(df.index), fill_value=0)
df

,5649565,5649585,5649607,5649630,5649646,5649648,5649659,5649662,5649677,5649679,...,5650692,5650704,5650743,5650746,5650752,5650760,5650772,5650777,5650783,5650785
5649565,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5649585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5649607,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5649630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5649646,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5650760,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,1,1
5650772,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,1
5650777,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,1,1
5650783,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,1,0,1


<b><i><u>Dropping the columns needed to be excluded having low relevancy score</u></i></b>

In [69]:
for i in data['customers'].unique():
    for j in data[data['customers'] == i].index:
        for k in data[ data['customers'] == i ].index:
            if(j not in data[data['customers'] == i].index or k not in data[data['customers'] == i].index):
                continue
            x=data.xs(j)['product']
            y=data.xs(k)['product']
            if (x in df.columns and y in df.columns and df.loc[x,y]==1):
                print(k)
                data.drop(index=k,inplace=True)

79
98
95
96
86
94
97
47
87
10
23
19
31
37
13
17
93
70
62
61
50
18
15
28
8
40
39
68
59
4
5
29
21
26
92
36
1
9
42
72
191
186
113
145
153
202
199
198
200
190
201
192
196
108
151
119
137
112
127
123
109
131
175
161
170
162
163
189
121
126
129
140
107
139
197
114
118
128
147
167
155
144
174
136
100
104
294
286
275
270
260
296
251
209
223
213
219
236
210
230
217
284
301
298
300
297
299
289
263
271
250
262
215
227
234
243
267
254
259
283
221
225
228
247
295
208
204
238
235
203
257
244
274
364
366
354
362
315
382
389
386
385
372
387
378
388
365
348
307
346
327
311
324
321
331
337
314
329
322
326
319
316
328
309
303
339
312
359
336
302
343
310
342
462
402
455
489
491
490
480
482
492
488
460
439
451
450
463
400
440
405
485
428
392
399
481
486
433
434
446
437
391
424
448
395
465
401
404
411
421
427
414
417
406
425
418
412
416
415
396
472
505
508
518
559
542
499
501
555
504
578
500
513
510
521
527
503
507
496
506
575
512
516
519
530
494
529
569
585
582
584
581
583
574
544
535
534
561
558
495
526
4

3858
3850
3855
3883
3832
3898
3903
3865
3836
3864
3833
3869
3920
3870
3881
3930
3967
3991
3978
3988
4007
3938
4010
3933
3974
3940
3959
3939
3953
3999
4017
4014
4015
4006
4016
4013
3927
3963
3962
3951
3954
3950
3952
3934
3949
3945
3955
3961
3937
3990
3929
3993
3926
3958
4005
3976
3968
3971
4085
4020
4057
4056
4027
4067
4095
4109
4110
4102
4111
4112
4108
4028
4041
4031
4038
4049
4055
4035
4070
4080
4068
4061
4034
4066
4047
4040
4044
4036
4046
4033
4106
4075
4022
4087
4054
4052
4060
4105
4098
4023
4063
4170
4125
4116
4118
4141
4113
4121
4165
4119
4152
4115
4172
4168
4124
4185
4161
4169
4151
4160
4133
4130
4127
4139
4123
4120
4136
4146
4156
4145
4134
4128
4132
4201
4273
4259
4202
4219
4213
4238
4229
4278
4198
4252
4270
4289
4286
4288
4285
4287
4277
4247
4248
4276
4197
4200
4207
4215
4221
4212
4210
4191
4255
4261
4190
4224
4223
4269
4283
4260
4189
4218
4196
4236
4228
4232
4192
4332
4329
4347
4348
4294
4338
4337
4362
4300
4367
4374
4371
4373
4370
4363
4372
4365
4297
4328
4364
4321
4311
4306


7543
7535
7526
7512
7504
7530
7492
7484
7489
7474
7521
7539
7550
7547
7548
7538
7549
7546
7540
7500
7471
7499
7491
7496
7481
7497
7466
7537
7524
7468
7527
7503
7631
7564
7629
7561
7609
7596
7590
7559
7594
7571
7575
7574
7627
7570
7579
7568
7583
7563
7576
7573
7566
7552
7585
7584
7577
7569
7582
7555
7587
7554
7604
7615
7593
7592
7556
7683
7684
7699
7692
7635
7637
7658
7657
7696
7671
7680
7661
7652
7647
7650
7695
7667
7638
7632
7654
7669
7662
7642
7644
7651
7675
7634
7686
7704
7776
7739
7714
7712
7774
7736
7706
7735
7772
7707
7745
7725
7708
7721
7711
7718
7729
7716
7771
7709
7757
7717
7713
7726
7727
7719
7724
7703
7753
7763
7740
7750
7826
7785
7834
7801
7816
7778
7795
7794
7833
7783
7802
7839
7836
7837
7829
7838
7835
7823
7811
7805
7817
7808
7814
7777
7791
7798
7911
7852
7909
7906
7854
7882
7880
7876
7848
7875
7855
7849
7851
7858
7869
7862
7865
7853
7866
7872
7860
7864
7867
7873
7841
7845
7910
7844
7895
7887
7999
7924
7988
7986
8001
8003
8004
8000
8002
7991
7992
7919
7958
7936
7932
7929


10983
11001
11029
11019
10982
10995
10990
11007
10991
10986
10989
10980
10997
10996
11018
10978
11021
11002
11011
11037
11074
11049
11053
11055
11064
11044
11086
11040
11051
11043
11048
11056
11058
11054
11076
11068
11073
11066
11072
11063
11075
11060
11039
11059
11149
11100
11092
11157
11127
11095
11089
11119
11118
11142
11152
11126
11137
11097
11105
11110
11112
11129
11122
11111
11096
11107
11103
11114
11117
11099
11146
11091
11134
11143
11166
11226
11216
11233
11230
11231
11223
11232
11229
11193
11186
11209
11199
11205
11197
11200
11190
11201
11206
11162
11191
11207
11168
11188
11171
11167
11177
11176
11163
11175
11165
11172
11178
11181
11227
11185
11267
11242
11277
11243
11247
11250
11258
11275
11290
11287
11288
11281
11289
11286
11238
11237
11269
11266
11255
11262
11253
11252
11240
11257
11244
11251
11234
11345
11300
11337
11312
11297
11311
11348
11352
11338
11303
11301
11306
11343
11358
11355
11357
11354
11356
11349
11336
11330
11315
11325
11291
11308
11335
11294
11339
11321
1131

14030
14048
14053
14033
14084
14126
14079
14123
14082
14097
14112
14113
14093
14103
14122
14089
14081
14088
14111
14078
14114
14090
14185
14139
14150
14136
14148
14138
14146
14153
14156
14180
14174
14178
14187
14141
14163
14171
14135
14165
14137
14176
14144
14140
14151
14129
14134
14157
14190
14162
14194
14235
14200
14196
14217
14216
14232
14226
14229
14218
14224
14237
14236
14198
14228
14240
14197
14199
14204
14212
14215
14206
14209
14211
14205
14208
14210
14214
14203
14245
14290
14253
14304
14289
14296
14311
14308
14309
14301
14310
14307
14278
14271
14270
14300
14250
14286
14248
14276
14266
14247
14265
14295
14255
14259
14249
14252
14256
14261
14257
14244
14291
14282
14355
14321
14333
14316
14348
14322
14354
14368
14365
14367
14364
14366
14358
14313
14331
14330
14357
14338
14334
14324
14326
14327
14339
14345
14347
14314
14341
14315
14415
14378
14420
14372
14392
14391
14374
14398
14396
14417
14399
14397
14386
14375
14383
14377
14381
14388
14390
14382
14385
14387
14371
14403
14409
1443

...

35485
35524
35512
35486
35482
35491
35484
35488
35494
35499
35487
35480
35501
35500
35506
35503
35505
35497
35498
35557
35554
35551
35550
35549
35567
35556
35533
35566
35546
35542
35530
35532
35536
35540
35538
35537
35634
35628
35633
35615
35619
35607
35578
35592
35569
35590
35589
35617
35601
35574
35616
35630
35610
35605
35602
35598
35631
35588
35568
35579
35583
35577
35655
35648
35651
35640
35663
35653
35654
35647
35650
35645
35671
35701
35693
35677
35707
35679
35674
35676
35680
35687
35685
35682
35691
35673
35690
35698
35675
35670
35703
35700
35681
35684
35686
35702
35729
35720
35743
35732
35737
35719
35714
35726
35744
35739
35736
35730
35741
35738
35788
35782
35792
35767
35791
35790
35806
35765
35749
35764
35805
35753
35772
35750
35773
35801
35751
35786
35797
35810
35802
35800
35808
35812
35811
35759
35754
35763
35775
35779
35860
35855
35849
35874
35839
35844
35820
35872
35873
35836
35841
35856
35843
35850
35851
35818
35834
35833
35866
35859
35828
35821
35824
35878
35898
35875
3589

38647
38602
38637
38641
38606
38619
38595
38618
38629
38600
38623
38630
38620
38628
38635
38597
38643
38646
38657
38652
38651
38658
38655
38659
38617
38614
38612
38601
38604
38609
38615
38645
38684
38677
38664
38667
38671
38680
38674
38710
38694
38661
38702
38707
38711
38668
38705
38714
38708
38715
38712
38716
38709
38683
38660
38665
38697
38676
38673
38678
38692
38690
38686
38747
38718
38755
38740
38766
38764
38736
38721
38723
38729
38734
38731
38732
38742
38720
38765
38724
38743
38745
38725
38735
38728
38726
38767
38770
38789
38811
38799
38771
38774
38816
38785
38775
38787
38817
38800
38804
38815
38784
38783
38772
38773
38781
38786
38788
38792
38871
38849
38821
38842
38867
38870
38855
38818
38862
38872
38852
38857
38845
38851
38836
38826
38839
38827
38840
38835
38834
38822
38824
38831
38837
38866
38936
38915
38906
38927
38947
38934
38959
38948
38942
38950
38960
38957
38961
38930
38919
38875
38937
38925
38935
38923
38940
38917
38922
38955
38888
38895
38885
38880
38897
38887
38894
3889

41629
41625
41621
41611
41622
41624
41655
41635
41654
41639
41648
41644
41647
41642
41638
41651
41707
41703
41706
41708
41694
41658
41700
41699
41691
41696
41704
41685
41690
41667
41677
41675
41673
41662
41665
41671
41676
41669
41678
41656
41661
41679
41682
41736
41746
41713
41728
41727
41744
41750
41743
41717
41762
41741
41748
41754
41733
41735
41726
41721
41714
41716
41719
41722
41720
41709
41724
41757
41730
41783
41767
41778
41773
41774
41763
41775
41786
41772
41771
41768
41829
41832
41833
41807
41822
41790
41825
41789
41793
41805
41834
41796
41831
41809
41812
41821
41814
41800
41797
41801
41788
41803
41868
41869
41866
41862
41844
41841
41852
41845
41851
41848
41856
41838
41854
41839
41853
41850
41847
41873
41840
41872
41867
41864
41836
41875
41892
41891
41924
41915
41923
41902
41880
41905
41903
41886
41883
41881
41882
41920
41907
41876
41917
41894
41921
41975
41976
41973
41927
41954
41930
41953
41990
41972
41964
41985
41961
41931
41963
41966
41939
41945
41936
41980
41989
41935
4193

44667
44635
44676
44659
44633
44638
44651
44649
44642
44650
44661
44663
44665
44685
44707
44688
44689
44694
44698
44696
44700
44695
44691
44699
44697
44690
44706
44702
44727
44718
44715
44763
44760
44751
44708
44754
44753
44761
44765
44758
44764
44741
44744
44732
44713
44717
44726
44723
44711
44720
44729
44728
44725
44721
44767
44781
44778
44768
44780
44769
44772
44771
44820
44823
44789
44791
44801
44814
44821
44803
44806
44818
44815
44812
44807
44800
44788
44844
44854
44828
44859
44832
44847
44851
44831
44843
44842
44839
44838
44834
44849
44829
44868
44879
44865
44889
44893
44895
44866
44903
44902
44906
44869
44898
44911
44875
44878
44871
44881
44907
44914
44910
44909
44915
44912
44916
44969
44972
44965
44925
44968
44979
44985
44971
44951
44918
44922
44982
44923
44954
44941
44932
44929
44975
44937
44931
44950
44924
44927
44934
44944
44942
44939
44936
44987
44986
44949
44917
44953
44983
44989
45010
45006
44988
44991
44995
45008
44997
45009
45001
45013
45017
45037
45035
45033
45032
4504

47591
47577
47578
47587
47600
47593
47603
47598
47597
47604
47601
47605
47627
47624
47630
47633
47629
47634
47613
47615
47618
47617
47614
47608
47609
47612
47619
47620
47622
47644
47640
47635
47639
47638
47641
47642
47636
47690
47683
47650
47695
47657
47660
47670
47685
47687
47681
47684
47691
47689
47654
47672
47659
47669
47665
47656
47662
47649
47653
47676
47696
47671
47668
47664
47698
47700
47704
47706
47709
47701
47714
47699
47735
47738
47724
47715
47730
47729
47739
47734
47737
47791
47748
47782
47790
47785
47743
47759
47767
47766
47792
47769
47740
47775
47779
47795
47787
47786
47796
47793
47797
47749
47753
47752
47744
47746
47750
47755
47832
47816
47817
47800
47822
47831
47826
47799
47802
47810
47825
47830
47821
47827
47834
47824
47829
47836
47833
47837
47815
47814
47798
47808
47860
47839
47873
47872
47889
47879
47890
47883
47891
47888
47892
47880
47886
47855
47856
47859
47887
47863
47882
47870
47848
47847
47845
47857
47867
47915
47920
47922
47912
47914
47924
47925
47906
47897
4790

<u><i><b>Saving the dataframe to perform further operations</b></i></u>

In [206]:
data.to_csv('output.csv')

<u><i><b>Resetting index</b></i></u>

In [208]:
data=pd.read_csv('output.csv')
data.reset_index(inplace=True)
data.drop(columns=['index'],axis=1,inplace=True)

In [209]:
data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
data

,customers,product,relevancy_score
0,A10001,5650743,0.646916
1,A10001,5649965,0.608653
2,A10001,5649679,0.587336
3,A10001,5650455,0.581182
4,A10001,5650704,0.546101
...,...,...,...
26684,A10786,5650506,0.003051
26685,A10786,5649842,0.002633
26686,A10787,5649878,0.008794
26687,A10787,5650544,0.008170


<u><b><i>Dropping rows having more than one product of same kind for same customer</i></b></u>

In [91]:
for i in data['customers'].unique():
    for j in data[data['customers'] == i].index:
        for k in data[ data['customers'] == i ].index:
            if(j==k or j not in data[data['customers'] == i].index or k not in data[data['customers'] == i].index):
                continue
            x=data.xs(j)['product']
            y=data.xs(k)['product']
            if(x==y):
                print(k)
                data.drop(index=k,inplace=True)

<u><b><i>Sorting values in descending order of relevancy score</i></b></u>

In [210]:
data=data.sort_values(by=["product","relevancy_score"],ascending=[True,False])
data

,customers,product,relevancy_score
1146,A10023,5649565,0.361841
28,A10001,5649565,0.293978
2483,A10050,5649565,0.236443
4668,A10100,5649565,0.211975
4155,A10088,5649565,0.194030
...,...,...,...
5494,A10120,5650785,0.418733
1921,A10039,5650785,0.369641
1096,A10022,5650785,0.308132
3332,A10069,5650785,0.244113


<b><i><u>Loading the Products dataset</b></i></u>

In [135]:
product=pd.read_csv('Products.csv')
product

,product,volume
0,5650512,150
1,5650506,600
2,5649630,300
3,5650672,200
4,5650537,100
...,...,...
109,5649927,200
110,5650746,150
111,5649851,400
112,5650644,400


<b><i><u>Dropping all the rows having low relevancy score for all the products having occurences greater than volume</u></i></b>

In [211]:
for i in data['product'].unique():
    x=(data[data['product'] == i].count().unique())
    y=data[data['product']==i].index
    for j in product[product['product']==i].index:
        f=1
        while(x>product.xs(j)['volume']):
            data.drop(index=y[-f],inplace=True)
            print(y[-f])
            x-=1
            f+=1

21993
16433
16740
22959
7765
24794
7921
22500
18796
13126
6281
2597
5531
11311
26300
13887
2366
10202
13171
9966
5292
11397
26588
2018
6339
20151
24751
19536
23454
25408
18413
24235
24018
22189
20987
25013
25201
23186
25166
26045
25233
21872
24790
22466
22520
18570
22275
16701
22945
9711
14189
23350
19821
18261
19388
21691
26151
25641
20590
13620
22021
25105
25123
20704
10941
24063
22628
17614
17405
18529
16450
12169
19768
17192
20193
11837
18997
19204
18132
15502
17643
21594
17748
19615
25214
16866
19434
13317
14574
15827
6576
14379
19916
11249
9906
13979
23061
13524
18803
14479
19489
17347
22234
25274
16820
14055
21982
21766
18952
11788
14414
16224
19729
11150
20015
20881
19454
15142
11590
12777
8920
17842
12832
17135
14775
18724
760
6718
15606
25176
16119
8760
17880
15091
14694
15199
18420
14224
15404
17701
14125
21728
10274
23396
14083
12670
20919
7637
9571
24576
9347
13137
15154
10648
15356
10173
9776
12609
10368
16994
16072
7533
19278
10743
21091
9010
16243
16952
2140
8649
13396


5029
13885
20466
1631
20904
12935
6468
12504
975
5885
19825
21403
2209
24266
4441
24681
22892
872
20152
8677
24296
13459
15520
10950
22182
24429
3310
19683
107
15577
17863
9926
25514
12051
16631
22647
14149
12746
11466
8921
26033
9846
15962
2363
24764
25590
20839
21566
18438
25183
5259
9408
19078
567
12792
24488
19623
16833
12075
12254
1475
22290
16568
9293
20398
12439
16278
23400
10102
17154
11685
22082
8461
12590
24553
8778
8199
15218
1415
22204
5716
24863
25284
21413
24103
26646
26226
13999
11620
23572
22476
25958
26621
26411
26344
21673
24821
26389
13436
16542
16397
26395
26585
25637
19740
26290
25662
25909
22872
25956
22619
26197
26246
26018
25852
24408
25251
26547
24320
24303
21277
21826
24471
25387
25144
24590
24770
22545
26358
25888
23465
24053
23430
25024
19513
10106
25831
23741
24664
24367
18271
25445
21585
13465
23363
20696
22621
12468
25955
26141
19051
16480
25174
21918
14870
22661
18245
19194
26363
20131
23257
22063
19093
21946
25753
24795
16546
19339
10158
21631
24854
153

21344
26429
26335
24630
16474
25471
26355
25023
21004
25845
14627
25314
20654
25942
9859
19507
26535
14022
19089
18603
14205
23412
25616
25130
17030
24838
11754
23190
22528
17212
25238
22608
19690
26613
26444
14933
24451
16767
13225
8891
24715
22679
21252
10149
5489
14327
12134
24378
25738
22836
15923
24035
10200
6434
24112
12186
21201
17795
10910
14293
6905
11356
14243
11031
14831
12441
19175
12005
2361
5157
7159
19144
23138
18967
6948
25404
19853
13166
22313
15759
22025
22993
19980
3351
24759
10342
22778
12930
9177
23346
23750
13012
21506
25700
16420
3216
8775
2403
24091
14383
20523
23041
18179
4121
19531
21060
16695
24548
24229
17932
17964
23151
16270
20764
5254
19262
22913
21183
24022
25114
24304
13998
16716
25173
20614
26635
20080
21085
16545
15399
18876
20045
23813
18514
25576
13386
22506
16677
25949
22300
20284
26187
11852
19793
19370
22819
24750
25868
18641
12760
18549
22051
23463
18672
10921
17876
21343
19955
16344
24524
18002
17912
18835
14210
18054
16594
20442
16208
21217
11

19631
21920
15333
24734
21350
14635
16717
18517
16779
26364
21706
25507
21086
23634
26481
18880
26496
16549
20539
18200
22547
25686
22348
13389
22404
23974
23692
21045
19568
15450
25782
26642
19372
19999
26015
26190
26386
20285
23892
26570
20044
13962
24628
8993
23937
10157
23510
20125
13906
26125
15845
24465
23563
11851
24048
18242
8791
17689
25379
23420
15392
24390
12515
20909
13761
26448
14946
18671
21627
15683
15538
23708
19691
20206
11756
22791
17871
18152
26086
21208
14729
15183
22384
22735
11567
25741
19942
13075
23001
23661
22898
20984
22446
14302
10955
9969
15735
10836
24234
11612
17996
12983
5165
11900
21247
14155
14802
22651
25083
15150
8396
10489
16877
10066
10452
6815
6952
11304
18825
23539
13808
20956
7162
14008
9188
12806
14326
518
12127
4131
15225
823
21971
2954
5210
12007
15795
14833
24509
19416
6177
20342
7028
14668
4219
18903
7337
18731
23444
13108
22951
10588
23589
1724
24186
1863
12048
23829
12326
22834
22145
3917
2159
19292
4338
17155
25159
18036
23725
20831
23779

8334
5621
2897
6930
20324
18890
5758
2489
21875
19786
23687
25066
17760
11439
16510
21600
19298
23359
17999
15437
17482
22366
23810
23384
17621
24875
22786
15527
13117
25980
21489
21786
19737
16979
14534
16633
13631
24923
16388
14199
26050
24263
26118
24602
25871
24490
25997
25269
8021
26572
18263
25568
24070
20767
23047
24506
25498
22274
23643
17443
10526
7974
18684
18226
8585
25218
24582
26532
7881
26319
12364
25698
23014
18999
24778
21555
7269
26269
6460
25913
18434
7672
9739
8504
26366
8556
19714
22020
26149
20703
23747
14759
19973
18626
8105
5767
7056
6985
21314
22392
21026
7333
17929
18464
25031
7699
17139
22461
19915
8944
8174
22908
17572
3775
21732
25364
17004
17527
16932
8608
21097
23527
13726
25930
21291
5102
7236
16310
15986
21138
15289
19258
14129
23981
15647
14053
5937
15143
7406
18561
18401
20013
26441
19311
20331
18097
22070
4922
19020
26213
19199
16161
26416
10864
17841
13477
18015
17815
24892
24417
9017
22160
14088
15892
17656
14408
20087
6608
18124
14648
18799
21953
1

22697
9721
18871
23193
25356
23929
20409
25710
22448
26332
23082
21406
24314
16052
16232
25310
21259
24654
15740
23882
26447
25769
14018
22927
25649
13335
21515
23334
17908
22297
12374
11753
20503
11468
26177
17590
24807
16207
26628
20869
23449
13227
22047
26308
23408
19085
24492
21989
17830
23706
9107
11508
13586
20632
10777
21599
15588
14905
10291
19940
17165
26369
18826
24036
18236
26299
16839
23271
20603
8148
23494
22650
17094
20563
8315
8254
16175
15837
18633
22835
14989
21077
17382
24606
13942
25063
13114
20113
15065
21299
11608
11220
9682
15630
21893
22467
15348
23537
13427
14434
23685
9256
14200
19003
8346
20815
9138
14724
14291
11957
14386
12335
13882
11865
15522
24680
19718
12406
21487
14927
19390
21199
25232
8508
17248
12747
21376
14464
11432
26256
15120
7341
17329
12646
25899
15867
10510
9362
20273
14832
18508
16702
5355
11393
23098
8285
12798
19294
5986
3535
25424
6657
13498
15019
19494
13989
26048
4729
12693
22120
4893
24903
19146
11254
7567
22314
9471
11842
9796
8809
339

<i><b><u>Sorting the table in the ascending order of customers and then descending order of relevancy score for each customer</u></b></i>

In [212]:
data=data.sort_values(by=["customers","relevancy_score"],ascending=[True,False])
data

,customers,product,relevancy_score
0,A10001,5650743,0.646916
1,A10001,5649965,0.608653
2,A10001,5649679,0.587336
3,A10001,5650455,0.581182
4,A10001,5650704,0.546101
...,...,...,...
26679,A10786,5650048,0.007043
26683,A10786,5650670,0.003587
26684,A10786,5650506,0.003051
26686,A10787,5649878,0.008794


<u><i><b>Saving the dataframe to perform further operations</b></i></u>

In [213]:
data.to_csv('output.csv')

<i><b><u>Dropping all the rows having customers purchasing less than 3 products</u></b></i>

In [221]:
freq=data['customers'].value_counts()<3
for i in data['customers'].unique():
    if(freq[i]):
        for j in (data[data['customers']==i].index):
            data.drop(index=j,inplace=True)

<i><u><b>Displaying the data</b></u></i>

In [222]:
data

,customers,product,relevancy_score
0,A10001,5650743,0.646916
1,A10001,5649965,0.608653
2,A10001,5649679,0.587336
3,A10001,5650455,0.581182
4,A10001,5650704,0.546101
...,...,...,...
26671,A10785,5649897,0.018079
26675,A10785,5650506,0.004726
26679,A10786,5650048,0.007043
26683,A10786,5650670,0.003587


<i><b><u>Dropping all the rows having customers buying more than 8 products having low relevancy score</u></b></i>

In [227]:
for i in data['customers'].unique():
    x=(data[data['customers'] == i].count().unique())
    y=data[data['customers']==i].index
    f=1
    while(x>8):
        data.drop(index=y[-f],inplace=True)
        print(y[-f])
        x-=1
        f+=1

49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
16
15
14
13
12
11
10
9
8
109
108
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
161
159
158
157
156
155
154
152
151
150
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
209
208
207
206
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
264
263
262
261
260
259
258
257
256
254
253
251
250
249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
231
230
229
228
227
226
225
224
223
222
221
220
219
218
315
314
313
311
310
309
308
306
305
304
303
302
301
300
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
367
366
365
364
363
362
361
360
359
357
356
355
352
351
350
349
347
346
34

2484
2483
2482
2481
2480
2479
2478
2477
2476
2475
2474
2473
2471
2551
2550
2549
2548
2547
2546
2545
2543
2542
2541
2539
2538
2537
2536
2535
2534
2532
2531
2530
2529
2528
2527
2526
2525
2524
2523
2522
2521
2520
2519
2518
2517
2596
2594
2593
2592
2590
2589
2587
2586
2585
2584
2581
2580
2579
2578
2577
2576
2575
2574
2573
2571
2570
2569
2568
2567
2566
2565
2564
2563
2562
2561
2646
2645
2644
2643
2642
2641
2640
2638
2637
2636
2635
2634
2633
2632
2630
2629
2628
2627
2626
2625
2624
2623
2622
2621
2620
2619
2618
2617
2616
2615
2614
2613
2612
2611
2610
2609
2607
2694
2693
2692
2689
2688
2687
2686
2685
2684
2683
2681
2680
2679
2678
2676
2675
2674
2673
2672
2671
2670
2669
2668
2667
2666
2665
2664
2663
2662
2661
2660
2659
2658
2657
2656
2725
2724
2723
2722
2721
2720
2719
2718
2717
2716
2715
2714
2713
2712
2710
2709
2708
2707
2706
2705
2704
2703
2778
2777
2776
2775
2774
2773
2772
2771
2770
2768
2767
2766
2765
2764
2763
2762
2761
2760
2759
2758
2756
2755
2754
2753
2752
2751
2750
2749
2748
2747
2746


4948
4947
4944
4943
4942
4941
4940
4938
4937
4936
4935
4934
4933
4932
4929
4928
4927
4926
4925
4924
4921
4920
4918
4917
4916
4915
4914
4913
4912
4911
4910
4909
4908
4907
4906
4905
4980
4979
4978
4977
4974
4972
4971
4970
4969
4968
4967
4966
4965
4964
4963
4962
4961
4960
4959
4958
4957
5032
5031
5027
5026
5024
5022
5021
5019
5018
5017
5016
5015
5014
5012
5011
5010
5009
5008
5007
5006
5005
5004
5003
5002
5001
5000
4999
4998
4997
4996
4995
4993
4992
4991
5070
5069
5065
5064
5063
5062
5061
5060
5059
5058
5057
5056
5055
5054
5053
5052
5051
5050
5049
5048
5046
5045
5044
5117
5116
5115
5112
5111
5110
5108
5107
5106
5105
5104
5101
5098
5097
5096
5095
5094
5093
5092
5091
5090
5089
5088
5087
5086
5085
5084
5083
5082
5081
5080
5079
5166
5164
5163
5161
5159
5158
5156
5154
5153
5152
5150
5149
5147
5146
5145
5144
5143
5142
5141
5140
5139
5138
5137
5136
5134
5133
5132
5131
5130
5129
5128
5127
5220
5219
5217
5216
5215
5214
5209
5208
5205
5204
5203
5202
5201
5200
5199
5198
5197
5196
5194
5192
5191
5190


7525
7524
7574
7572
7571
7569
7568
7565
7563
7562
7561
7560
7559
7558
7557
7556
7555
7554
7553
7552
7551
7550
7608
7605
7604
7603
7602
7600
7599
7598
7596
7595
7594
7593
7592
7591
7590
7589
7588
7587
7586
7585
7584
7643
7641
7638
7636
7635
7634
7633
7632
7631
7630
7629
7628
7627
7626
7625
7624
7623
7622
7621
7619
7618
7678
7676
7675
7673
7671
7670
7669
7668
7667
7666
7665
7664
7663
7662
7661
7660
7659
7658
7657
7715
7711
7710
7709
7708
7707
7706
7705
7704
7702
7701
7698
7697
7696
7695
7694
7693
7692
7691
7690
7689
7688
7738
7737
7736
7735
7734
7733
7732
7731
7730
7729
7728
7727
7726
7725
7724
7766
7760
7759
7758
7756
7755
7754
7753
7752
7751
7750
7749
7794
7793
7792
7791
7790
7789
7788
7787
7785
7784
7783
7782
7781
7780
7779
7778
7777
7776
7775
7834
7833
7832
7831
7829
7827
7826
7825
7824
7823
7820
7817
7816
7815
7814
7813
7812
7811
7810
7855
7854
7853
7851
7850
7849
7848
7847
7846
7845
7844
7892
7890
7887
7886
7885
7884
7883
7882
7879
7878
7877
7876
7875
7874
7873
7872
7871
7870
7869


10556
10551
10550
10549
10548
10547
10546
10545
10544
10604
10601
10599
10598
10597
10596
10591
10590
10589
10587
10586
10582
10581
10580
10579
10578
10577
10576
10575
10574
10572
10571
10570
10569
10568
10567
10638
10636
10635
10634
10632
10631
10630
10629
10628
10627
10626
10625
10624
10622
10621
10620
10619
10618
10617
10616
10689
10687
10684
10682
10679
10677
10674
10672
10671
10669
10668
10667
10666
10665
10664
10663
10662
10661
10657
10655
10654
10653
10652
10736
10734
10732
10731
10730
10729
10728
10724
10723
10722
10721
10720
10719
10718
10716
10714
10713
10712
10710
10709
10708
10707
10706
10705
10704
10703
10702
10699
10791
10789
10787
10782
10781
10780
10779
10778
10775
10773
10770
10769
10768
10767
10766
10765
10764
10763
10762
10761
10758
10757
10756
10754
10753
10752
10751
10750
10749
10748
10747
10848
10847
10839
10837
10833
10832
10831
10830
10829
10828
10827
10826
10825
10824
10823
10822
10821
10820
10819
10818
10817
10816
10815
10813
10811
10810
10809
10808
10807
1080

13265
13264
13263
13261
13260
13259
13258
13256
13255
13254
13253
13252
13294
13293
13338
13334
13333
13332
13331
13328
13327
13325
13324
13322
13321
13320
13319
13318
13316
13315
13314
13313
13312
13311
13310
13309
13308
13391
13390
13388
13384
13381
13377
13376
13375
13374
13373
13372
13371
13368
13367
13366
13365
13364
13361
13360
13359
13358
13357
13356
13355
13354
13435
13433
13431
13428
13426
13424
13419
13418
13417
13416
13414
13413
13412
13411
13410
13409
13408
13407
13406
13405
13403
13463
13462
13461
13458
13457
13456
13455
13453
13452
13451
13508
13507
13501
13496
13495
13494
13493
13492
13491
13490
13489
13488
13487
13485
13484
13483
13482
13481
13480
13479
13478
13476
13475
13474
13549
13548
13545
13543
13540
13539
13538
13536
13535
13534
13533
13532
13530
13529
13528
13527
13526
13525
13523
13522
13521
13596
13595
13594
13592
13590
13587
13583
13581
13580
13579
13578
13577
13574
13573
13572
13571
13570
13569
13567
13566
13565
13564
13563
13562
13561
13648
13646
13643
1363

16318
16317
16316
16315
16314
16313
16312
16366
16364
16363
16362
16361
16360
16359
16358
16392
16391
16390
16389
16387
16386
16385
16384
16383
16382
16381
16379
16377
16376
16431
16428
16425
16424
16422
16419
16416
16415
16414
16413
16412
16411
16409
16408
16478
16476
16471
16469
16468
16465
16462
16461
16456
16454
16453
16452
16451
16449
16447
16446
16444
16443
16509
16508
16507
16505
16502
16501
16500
16498
16497
16496
16495
16494
16493
16544
16540
16538
16537
16536
16535
16533
16531
16530
16528
16527
16526
16525
16524
16523
16606
16605
16601
16599
16598
16597
16596
16591
16590
16589
16588
16586
16583
16576
16573
16572
16571
16570
16569
16567
16566
16564
16563
16562
16634
16632
16630
16629
16628
16627
16626
16625
16624
16623
16622
16621
16620
16618
16676
16675
16674
16671
16670
16667
16666
16665
16664
16662
16661
16659
16658
16657
16655
16654
16652
16651
16650
16649
16648
16647
16720
16710
16708
16705
16704
16703
16700
16699
16697
16696
16738
16736
16732
16731
16730
16729
16777
1677

20127
20122
20120
20119
20117
20114
20112
20111
20109
20108
20106
20105
20104
20103
20102
20101
20100
20097
20155
20150
20149
20147
20146
20145
20144
20143
20142
20175
20173
20171
20170
20169
20168
20167
20210
20208
20207
20205
20202
20200
20199
20198
20197
20247
20245
20243
20242
20238
20231
20230
20229
20228
20227
20226
20225
20224
20274
20272
20269
20268
20266
20265
20264
20263
20262
20261
20260
20259
20258
20321
20319
20317
20313
20311
20310
20308
20306
20305
20304
20302
20301
20300
20299
20297
20347
20346
20344
20341
20340
20338
20335
20334
20380
20376
20372
20370
20369
20367
20365
20364
20363
20362
20360
20359
20410
20408
20404
20403
20402
20401
20399
20397
20396
20394
20392
20443
20437
20435
20433
20432
20431
20430
20429
20479
20477
20473
20472
20471
20468
20467
20464
20463
20462
20461
20460
20458
20457
20501
20498
20497
20496
20495
20494
20492
20541
20536
20533
20532
20530
20529
20528
20527
20526
20524
20522
20521
20520
20579
20577
20576
20575
20574
20570
20568
20564
20561
2056

25326
25324
25319
25318
25316
25396
25390
25384
25382
25378
25428
25476
25474
25473
25469
25467
25527
25523
25522
25558
25556
25579
25575
25574
25573
25635
25634
25633
25631
25630
25628
25626
25625
25624
25690
25688
25687
25682
25680
25760
25759
25757
25755
25754
25787
25786
25785
25783
25781
25778
25853
25851
25850
25846
25892
25889
25886
25885
25908
25960
25959
25953
25952
25950
25946
25945
25985
25982
25979
26026
26017
26073
26071
26068
26067
26066
26145
26139
26201
26198
26196
26195
26225
26249
26243
26292
26349
26343
26342
26339
26338
26392
26412
26410
26439
26436
26435


<i><b><u>Making final changes as per submission format</u></b></i>

In [240]:
data.reset_index()
data=data.rename(columns = {"customers":"customers_org"}) 
data

,customers_org,product,relevancy_score
0,A10001,5650743,0.646916
1,A10001,5649965,0.608653
2,A10001,5649679,0.587336
3,A10001,5650455,0.581182
4,A10001,5650704,0.546101
...,...,...,...
26671,A10785,5649897,0.018079
26675,A10785,5650506,0.004726
26679,A10786,5650048,0.007043
26683,A10786,5650670,0.003587


<i><b><u>Saving the final dataframe to the output csv file</u></b></i>

In [241]:
data.to_csv('output.csv',index=False)

<i><b>Now all the constraints are satisfied and total relevancy is maximized which contains 6147 rows and 3 columns.</b></i>